In [73]:
import geopandas as gpd
import pandas as pd

# Venturilo - Contextual count embedder 

### Task 1

Load dataset with *venturilo* bike stations (`data/venturilo_stations.json`) and convert *lat/lon* into a geometry column. Save it to `stations_gdf` variable

<details>
<summary>Solution</summary>

```python
stations_raw = pd.read_json('data/venturilo_stations.json')
stations_gdf = gpd.GeoDataFrame(
    stations_raw,
    geometry=gpd.GeoSeries.from_xy(stations_raw["lon"], stations_raw["lat"]),
    crs="EPSG:4326",
)
stations_gdf.head()
```

</details>

In [ ]:
stations_gdf = ...
stations_gdf.head()

Downloading area of Warsaw in preparation for features download. Visualization of station location on the map

In [ ]:
from srai.regionalizers import geocode_to_region_gdf

warsaw_region = geocode_to_region_gdf("Warsaw, PL").to_crs("EPSG:4326")
m = warsaw_region.explore(tooltip=False, highlight=False, style_kwds={"fillOpacity": 0.3})
stations_gdf.explore(m=m, color="orange")

### Task 2
Split the area of Warsaw into regions, for which we will be predicting stations location

<details>
<summary>Solution</summary>

This is an example with H3 cells on resolution 9

```python
from srai.regionalizers import H3Regionalizer

regions_gdf = H3Regionalizer(resolution=9).transform(warsaw_region)
stations_gdf.head()
```

</details>

In [ ]:
from srai.plotting import plot_regions

regions_gdf = ...
plot_regions(regions_gdf)

### Task 3

Download the OSM tags which will be used to predict bicycle stations locations. But be honest, remove `"amenity": "bicycle_rental"` tag ;)

You can use one of our predefined filters or create any for your own

<details>
<summary>Solution</summary>

This is an example with H3 cells on resolution 9

```python
from srai.loaders import OSMPbfLoader

features_gdf = OSMPbfLoader().load(warsaw_region,GEOFABRIK_LAYERS)
features_gdf = features_gdf[features_gdf["shopping"] != "amenity=bicycle_rental"]
features_gdf.head()
```

</details>

In [ ]:
from srai.loaders.osm_loaders.filters import GEOFABRIK_LAYERS, HEX2VEC_FILTER, BASE_OSM_GROUPS_FILTER

features_gdf = ...
features_gdf.head()

Our features have not been associated with regions yet. We can use an *intersects* predicate and associate them with regions.

In [ ]:
from srai.joiners import IntersectionJoiner

joined_features = IntersectionJoiner().transform(regions_gdf, features_gdf)
joined_features

### Task 4

We have already accociated OSM features with regions. To train our model we have to join station locations with regions as well. Write the code which finds regions intersecting with station locations. Use those information to select positive and negative samples for classifier training (regions with and without stations). Remember that we wiil have to train model based on that, so make sure to do any neccessary undersampling to balance our training data

<details>
<summary>Solution</summary>

First, join bike stations locations with regions, using `IntersectionJoiner`
```python
bikes_joint = IntersectionJoiner().transform(regions_gdf, stations_gdf)
```

For future visualizations, we will need to restore geometry column
```python
positive_samples = regions_gdf.join(bikes_joint, how="inner")
positive_samples = positive_samples.reset_index().drop(columns=["feature_id"]).groupby("region_id").agg("first")  # this one is to remove duplicates
positive_samples = positive_samples.reset_index().set_index("region_id")
positive_samples["is_positive"] = True
```

Mark remaining regions as negative
```python
negative_samples = regions_gdf.copy()
negative_samples["is_positive"] = False
negative_samples.loc[positive_samples.index, "is_positive"] = True
negative_samples = negative_samples[~negative_samples["is_positive"]]
```

Just to keep everything balanced - undersampling
```python
negative_samples = negative_samples.sample(n=3 * len(positive_samples), random_state=42)
```

</details>

In [ ]:
positive_samples = ...
negative_samples = ...

train_data = pd.concat([positive_samples, negative_samples])
train_data.explore("is_positive", cmap="cividis", zoom_start=13, tiles="CartoDB positron")

Let's create embeddings for each region in our city (embeddings form outside of training data will be used for visualizations). Those will serve as our *Xs* for training, and *Ys* will be binary value if station is in the area or not

In [141]:
from srai.embedders import ContextualCountEmbedder
from srai.neighbourhoods import H3Neighbourhood

embedder = ContextualCountEmbedder(neighbourhood=H3Neighbourhood(), neighbourhood_distance=5, concatenate_vectors=True)
embeddings = embedder.transform(regions_gdf=regions_gdf, features_gdf=features_gdf, joint_gdf=joined_features)
X = embeddings.loc[train_data.index].to_numpy()
Y = train_data["is_positive"].astype(int).to_numpy()

### Task 5

Select your favourite model and train a classifier for station locations

<details>
<summary>Solution</summary>

```python
from sklearn.svm import SVC

classifier = SVC(probability=True)
classifier.fit(X_train, Y_train)
y_pred = classifier.predict(X_test)
y_pred_proba = classifier.predict_proba(X_test)

print(classification_report(Y_test, Y_pred))
```

</details>

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=42)

Y_pred = ...

print(classification_report(Y_test, Y_pred))

### Task 6

Run predictions for all regions and prepare visualization on the map

<details>
<summary>Solution</summary>

```python
from srai.plotting import plot_numeric_data

station_probas = classifier.predict_proba(embeddings.to_numpy())
regions_gdf["station_proba"] = station_probas[:, 1]
m = plot_numeric_data(regions_gdf, "station_proba", colormap="Spectral_r", opacity=0.5)
stations_gdf.explore(m=m, color='black')
```
</details>

In [ ]:
...

### Final task - transfer learning

Now we have a model, which was trained on data from Warsaw. Select some other city, and run predictions on it. Let's see where to put BSS stations there

<details>
<summary>Solution</summary>

```python
# Select area
wroclaw_region = geocode_to_region_gdf('Wrocław, PL').to_crs('EPSG:4326')

# Split into regions
wroclaw_regions_gdf = H3Regionalizer(resolution=9).transform(wroclaw_region)

# Load OSM features (the same as for model training)
wroclaw_features_gdf = OSMPbfLoader().load(wroclaw_region,GEOFABRIK_LAYERS)
wroclaw_features_gdf = wroclaw_features_gdf[wroclaw_features_gdf["shopping"] != "amenity=bicycle_rental"]

# Wrocław has a BSS, so let's download station locations for reference
from srai.loaders import OSMOnlineLoader
wroclaw_stations = OSMOnlineLoader().load(wroclaw_region, {"amenity": "bicycle_rental"})

# Get embeddings for regions
wroclaw_joined_features = IntersectionJoiner().transform(wroclaw_regions_gdf, wroclaw_features_gdf)
wroclaw_embeddings = embedder.transform(
    regions_gdf=wroclaw_regions_gdf,
    features_gdf=wroclaw_features_gdf,
    joint_gdf=wroclaw_joined_features,
)

# Predict and visualize
station_probas = classifier.predict_proba(wroclaw_embeddings.to_numpy())

wroclaw_regions_gdf["station_proba"] = station_probas[:, 1]
m = plot_numeric_data(wroclaw_regions_gdf, "station_proba", colormap="Spectral_r", opacity=0.5)

wroclaw_stations.explore(m=m, color='black')
```
</details>

In [ ]:
...